In [1]:
TRAIN_PATH               = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/preprocessed_category_train_.csv"
EVAL_PATH                = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/preprocessed_category_evalset.csv"
TEST_PATH                = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/preprocessed_category_testset.csv"
ALL_USER_FEATURES   = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status"] 

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas(leave=False)

class BlackFridayTopKBaseline:
    def __init__(self, trainset_location, user_features_list, build_inplace=False):
        self.trainset_location, self.user_features_list = trainset_location, user_features_list
        if build_inplace:
            self.build()
        
    def build(self):
        self.products = pd.read_csv(self.trainset_location)["Product_ID"].value_counts().index
        
    def predict(self, user_features, n_products):
        return self.products[:n_products]

In [4]:
df = pd.read_csv(TEST_PATH)
df_grouped = df.groupby(ALL_USER_FEATURES)["Product_ID"].apply(list).apply(np.array).reset_index()
top_k = [1, 10, 50, 100]
results = dict()

baseline = BlackFridayTopKBaseline(TRAIN_PATH, ALL_USER_FEATURES, True)
for k in tqdm(top_k, leave=False):
    metrics_id = f"baseline_top{k}"
    df_grouped[metrics_id]         = df_grouped[ALL_USER_FEATURES].progress_apply(lambda x: baseline.predict(x, k), axis=1)
    baseline_true_positives        = df_grouped.progress_apply(lambda x: np.intersect1d(x["Product_ID"],x[metrics_id]).shape[0], axis=1).sum()
    baseline_false_negatives       = df.shape[0] - baseline_true_positives
    baseline_false_positives       = df_grouped[metrics_id].apply(lambda x: x.shape[0]).sum() - baseline_true_positives
    baseline_reach                 = df_grouped.progress_apply(lambda x: min(x["Product_ID"].shape[0],x[metrics_id].shape[0]), axis=1).sum()
    results[f"baseline_top{k}"]    = {
        "top"  : k,
        "model": "baseline",
        "tp"   : baseline_true_positives,
        "fp"   : baseline_false_positives,
        "fn"   : baseline_false_negatives,
        "reach": baseline_reach #portata
    }

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

...

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

In [5]:
df_results = pd.DataFrame(results).T
df_results["precision"] = df_results["tp"]/(df_results["tp"]+df_results["fp"])
df_results["recall"] = df_results["tp"]/(df_results["tp"]+df_results["fn"])
df_results["tp_over_reach"] = df_results["tp"]/df_results["reach"]

In [6]:
df_results.pivot("model","top","precision")

top,1,10,50,100
model,,,,
baseline,0.260082,0.220165,0.183481,0.156519


In [7]:
df_results.pivot("model","top","recall")

top,1,10,50,100
model,,,,
baseline,0.002304,0.019505,0.081277,0.138666


In [8]:
df_results.pivot("model","top","tp_over_reach")

top,1,10,50,100
model,,,,
baseline,0.260082,0.22658,0.242582,0.259921


# another baseline

In [15]:
class BlackFridayRandomBaseline:
    def __init__(self, trainset_location, user_features_list, build_inplace=False):
        self.trainset_location, self.user_features_list = trainset_location, user_features_list
        if build_inplace:
            self.build()
        
    def build(self):
        self.products = pd.read_csv(self.trainset_location)["Product_ID"].value_counts().index
        np.random.seed(42)
        
    def predict(self, user_features, n_products):
        return self.products[np.random.choice(len(self.products), size=n_products, replace=False)]

In [16]:
df = pd.read_csv(TEST_PATH)
df_grouped = df.groupby(ALL_USER_FEATURES)["Product_ID"].apply(list).apply(np.array).reset_index()
top_k = [1, 10, 50, 100]
results = dict()

baseline = BlackFridayRandomBaseline(TRAIN_PATH, ALL_USER_FEATURES, True)
for k in tqdm(top_k, leave=False):
    metrics_id = f"baseline_top{k}"
    df_grouped[metrics_id]         = df_grouped[ALL_USER_FEATURES].progress_apply(lambda x: baseline.predict(x, k), axis=1)
    baseline_true_positives        = df_grouped.progress_apply(lambda x: np.intersect1d(x["Product_ID"],x[metrics_id]).shape[0], axis=1).sum()
    baseline_false_negatives       = df.shape[0] - baseline_true_positives
    baseline_false_positives       = df_grouped[metrics_id].apply(lambda x: x.shape[0]).sum() - baseline_true_positives
    baseline_reach                 = df_grouped.progress_apply(lambda x: min(x["Product_ID"].shape[0],x[metrics_id].shape[0]), axis=1).sum()
    results[f"baseline_top{k}"]    = {
        "top"  : k,
        "model": "baseline",
        "tp"   : baseline_true_positives,
        "fp"   : baseline_false_positives,
        "fn"   : baseline_false_negatives,
        "reach": baseline_reach #portata
    }

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

...

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

In [17]:
df_results = pd.DataFrame(results).T
df_results["precision"] = df_results["tp"]/(df_results["tp"]+df_results["fp"])
df_results["recall"] = df_results["tp"]/(df_results["tp"]+df_results["fn"])
df_results["tp_over_reach"] = df_results["tp"]/df_results["reach"]

In [18]:
df_results.pivot("model","top","precision")

top,1,10,50,100
model,,,,
baseline,0.027984,0.028148,0.028617,0.028934


In [19]:
df_results.pivot("model","top","recall")

top,1,10,50,100
model,,,,
baseline,0.000248,0.002494,0.012677,0.025634


In [20]:
df_results.pivot("model","top","tp_over_reach")

top,1,10,50,100
model,,,,
baseline,0.027984,0.028968,0.037835,0.048049
